In [31]:
import multiprocessing as mp
from numba import njit
import numpy as np
import time
import concurrent.futures as cf
import mkl
import time
mkl.set_num_threads(12)

1

In [66]:
# @njit
def square_sum(x):
    out = (np.tanh(1/x.dot(x)) ** 2)
#     out = out * out
    out = out.mean()
    return out
def cal_time(x):

    s=time.time()
    out = square_sum(x)
    print(time.time()-s)
    print(out)
#     return out

def run(x,queue):
    
    queue.put(cal_time(x))

def square_sum_mp(x):
    s=time.time()
    p = mp.Pool(6)
    p.map(cal_time, [x for _ in range(6)])
    p.close()
    p.join()
    print('time is',time.time()-s,'\n')

In [89]:
x = np.random.rand(1*10**6).reshape(1000,1000).astype(np.float32)
# y = np.random.rand(1*10**4).astype(np.float32)
out = cal_time(x)

0.025615692138671875
1.6041871e-05


In [91]:
import dask.array as da
x_ = da.from_array(x, chunks=(100, 1000))
x_

dask.array<array, shape=(1000, 1000), dtype=float32, chunksize=(100, 1000), chunktype=numpy.ndarray>

In [87]:
%time (da.tanh(1/x_.dot(x_)) ** 4).mean().compute()

CPU times: user 627 ms, sys: 1.19 s, total: 1.81 s
Wall time: 98.8 ms


/home/keisuke/miniconda3/envs/research/lib/python3.8/site-packages/dask/array/routines.py:294: PerformanceWarning: Increasing number of chunks by factor of 10
  intermediate = blockwise(


2.577564e-10

In [61]:
%time x.dot(x)

CPU times: user 4.77 s, sys: 1.31 s, total: 6.08 s
Wall time: 525 ms


array([[1009.30096, 1006.0159 , 1008.95886, ..., 1006.2321 , 1010.52075,
        1016.31866],
       [ 998.88354, 1017.28394, 1024.0021 , ..., 1002.0606 , 1017.4078 ,
        1017.41754],
       [1000.304  ,  996.6962 , 1009.10376, ...,  993.8533 ,  997.119  ,
        1011.1522 ],
       ...,
       [1014.13495, 1015.78894, 1023.7437 , ..., 1004.4581 , 1007.6599 ,
        1016.9664 ],
       [1013.69196, 1020.8745 , 1020.76044, ..., 1010.88116, 1002.4144 ,
        1022.6471 ],
       [1002.2323 ,  999.4808 , 1002.795  , ...,  991.05597,  995.671  ,
        1013.16284]], dtype=float32)

In [45]:
cal_time(x)

0.02089715003967285
2.5913752e-10


In [40]:
x = np.random.rand(1*10**6).reshape(1000,1000).astype(np.float32)
# y = np.random.rand(1*10**4).astype(np.float32)
out = cal_time(x)

0.025740385055541992
2.5913752e-10


In [30]:
queue = []
process = []
for i in range(6):
    q = mp.Queue()
    process.append(mp.Process(target=run, args = (x,q)))
    queue.append(q)
    process[i].start()
    
try:
    for worker in process:
        worker.join()
except KeyboardInterrupt:
    print('parent received keyboardinterrupt')
    for worker in process:
        worker.terminate()
        worker.join()
for q in queue:
    print(q.get())
#     print(queue.get())

0.067096471786499020.0696563720703125

2.5873184150704276e-102.5873184150704276e-100.06612253189086914


2.5873184150704276e-10
0.061064720153808594
2.5873184150704276e-10
0.06978273391723633
0.06823754310607912.5873184150704276e-10

2.5873184150704276e-10
None
None
None
None
None
None


In [25]:
while not queue.empty():
    print(queue.get())

1.9102422589624461
1.9102422589624461
1.9102422589624461
1.9102422589624461
1.9102422589624461
1.9102422589624461


In [24]:
queue.empty()

False

In [3]:
wrapper_func = time_wrapper(f= square_sum)

In [4]:
wrapper_func(np.random.rand(5*10**3), np.random.rand(5*10**3))

0.00017142295837402344


1.9163098336184357

In [8]:


with cf.ProcessPoolExecutor() as executor:
    
    args = ((x,y) for _ in range(10))
    
    results = executor.map(square_sum, args)
    
#     results = [executor.submit(square_sum, [x, y]) for _ in range(10)]
    
    for f in cf.as_completed(results):
        print(f.result())


TypeError: square_sum() missing 1 required positional argument: 'y'

In [15]:
import multiprocessing, os, time

def do_work():
    print('Work Started: %d' % os.getpid())
    time.sleep(2)
    return 'Success'

def manual_function(job_queue, result_queue):
    signal.signal(signal.SIGINT, signal.SIG_IGN)
    while not job_queue.empty():
        try:
            job = job_queue.get(block=False)
            result_queue.put(do_work())
        except Queue.Empty:
            pass
        #except KeyboardInterrupt: pass

def main():
    job_queue = multiprocessing.Queue()
    result_queue = multiprocessing.Queue()

    for i in range(6):
        job_queue.put(None)

    workers = []
    for i in range(3):
        tmp = multiprocessing.Process(target=manual_function,
                                      args=(job_queue, result_queue))
        tmp.start()
        workers.append(tmp)

    try:
        for worker in workers:
            worker.join()
    except KeyboardInterrupt:
        print('parent received ctrl-c')
        for worker in workers:
            worker.terminate()
            worker.join()

    while not result_queue.empty():
        print(result_queue.get(block=False))

In [ ]:
main

In [ ]:
square_sum(np.random.rand(10))

In [ ]:
@njit
def dype_(x):
    y = x.astype(np.int8)
    
    return y

In [ ]:
queue = mp.Queue(6)
for _ in range(6):
    process = mp.Process(target=run_, args=(np.random.rand(5*10**7).astype(np.float16),queue))
    process.daemon = True
    process.start() 
    try:
        process.join()
    except KeyboardInterrupt:
        print('wait for worker to cleanup...')
        queue.put('exit')
        process.join()

In [ ]:
process.start()

try:
    process.join()
except KeyboardInterrupt:
    print('wait for worker to cleanup...')
    queue.put('exit')
    process.join()

In [ ]:
import queue
import time
def test04():
    q = queue.Queue(10)
    for x in range(10):
        q.put(x)

    while q.not_empty:
        print('content--->',q.get())
        time.sleep(1)
    re = q.task_done()  
    print('state--->',re,'\n')
    q.join()
    print('over \n')

test04()

In [ ]:
import multiprocessing as mp

In [ ]:
mp.Queue